# Manipulating Spatial Data

* arcpy.da cursors
* Search cursor
* Update cursor
* Insert cursor
* SQL query

## Test Data
* http://spatial.binghamton.edu/geog503/data/lecture7_data.zip

## What is Cursor?

* Similar to list functions:
    - Iterate through feature classes, tables, and fields
* Cursors:
    - Iterate through rows in a table
    - Read and write geometries to and from records, row by row

![](https://wetlands.io/file/teaching/geog-503/images/cursors.png)


## Cursor Methods

* <http://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-classes/cursor.htm>

![](https://wetlands.io/file/teaching/geog-503/images/cursor_methods.png)

## General Syntax

* **Required parameters:**
    - An input table
    - A list (or tuple) of field names
* **General syntax:**
    - arcpy.da.InsertCursor(in_table, field_names)
    - arcpy.da.SearchCursor(in_table, field_names, {where_clause}, {spatial_reference}, {explore_to_points})
    - arcpy.da.UpdateCursor(in_table, field_names, {where_clause}, {spatial_reference}, {explore_to_points})


## Using the SearchCursor 

* **General syntax:**
    - arcpy.da.SearchCursor(in_table, field_names, {where_clause}, {spatial_reference}, {explore_to_points})
    - <http://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-data-access/searchcursor-class.htm>
    
* **Parameters:**

![](https://wetlands.io/file/teaching/geog-503/images/search_cursor_params.png)


* **Methods:**

![](https://wetlands.io/file/teaching/geog-503/images/search_cursor_methods.png)

In [1]:
try:
    import archook #The module which locates arcgis
    archook.get_arcpy()
    import arcpy
except ImportError:
    print("import arcpy error")

In [2]:
# Get column names (fields) from attribute table
import arcpy
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
desc = arcpy.Describe(fc)
fields = desc.fields
for field in fields:
    print("{:15} {}".format(field.name, field.type))

FID             OID
Shape           Geometry
AREA            Double
PERIMETER       Double
AIRPRTX020      Double
LOCID           String
FEATURE         String
NAME            String
TOT_ENP         Double
STATE           String
COUNTY          String
FIPS            String
STATE_FIPS      String


In [ ]:
# print all rows of selected columns
import arcpy
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
fields = ["LOCID", "NAME"]
cursor = arcpy.da.SearchCursor(fc, fields)
for row in cursor:
    print("LOCID = {}, NAME = {}".format(row[0], row[1]))
del cursor, row

In [4]:
# print only a few rows
import arcpy
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
fields = ["LOCID", "NAME"]
cursor = arcpy.da.SearchCursor(fc, fields)
for index, row in enumerate(cursor):
    print("Index = {}, LOCID = {}, NAME = {}".format(index, row[0], row[1]))
    if index == 10: 
        break
del cursor, row

Index = 0, LOCID = 4Z7, NAME = Hyder
Index = 1, LOCID = KCL, NAME = Chignik Lagoon
Index = 2, LOCID = KKA, NAME = Koyuk
Index = 3, LOCID = KVL, NAME = Kivalina
Index = 4, LOCID = 5KE, NAME = Ketchikan Harbor
Index = 5, LOCID = MTM, NAME = Metlakatla
Index = 6, LOCID = KWF, NAME = Waterfall
Index = 7, LOCID = KTN, NAME = Ketchikan
Index = 8, LOCID = KXA, NAME = Kasaan
Index = 9, LOCID = HYL, NAME = Hollis
Index = 10, LOCID = CGA, NAME = Craig


In [5]:
# Select by attribute
import arcpy
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
fields = ["LOCID", "NAME", "FEATURE"]
cursor = arcpy.da.SearchCursor(fc, fields, where_clause="\"FEATURE\" <> 'Airport'")
for row in cursor:
    print("LOCID = {:10} NAME = {:20} FEATURE = {}".format(row[0], row[1], row[2]))
del cursor, row

LOCID = 4Z7        NAME = Hyder                FEATURE = Seaplane Base
LOCID = 5KE        NAME = Ketchikan Harbor     FEATURE = Seaplane Base
LOCID = MTM        NAME = Metlakatla           FEATURE = Seaplane Base
LOCID = KWF        NAME = Waterfall            FEATURE = Seaplane Base
LOCID = KXA        NAME = Kasaan               FEATURE = Seaplane Base
LOCID = HYL        NAME = Hollis               FEATURE = Seaplane Base
LOCID = CGA        NAME = Craig                FEATURE = Seaplane Base
LOCID = KTB        NAME = Thorne Bay           FEATURE = Seaplane Base
LOCID = KCC        NAME = Coffman Cove         FEATURE = Seaplane Base
LOCID = 84K        NAME = Meyers Chuck         FEATURE = Seaplane Base
LOCID = 78K        NAME = Yes Bay Lodge        FEATURE = Seaplane Base
LOCID = KAE        NAME = Kake                 FEATURE = Seaplane Base
LOCID = AGN        NAME = Angoon               FEATURE = Seaplane Base
LOCID = FNR        NAME = Funter Bay           FEATURE = Seaplane Base
LOCID 

**With** statement:
* Guarantee closure and release of database lock
* Reset iteration


In [6]:
import arcpy
import arcpy.da
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
fields = ["LOCID", "NAME"]
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for index, row in enumerate(cursor):
        print("LOCID = {}, NAME = {}".format(row[0], row[1]))
        if index == 10:
            break

LOCID = 4Z7, NAME = Hyder
LOCID = KCL, NAME = Chignik Lagoon
LOCID = KKA, NAME = Koyuk
LOCID = KVL, NAME = Kivalina
LOCID = 5KE, NAME = Ketchikan Harbor
LOCID = MTM, NAME = Metlakatla
LOCID = KWF, NAME = Waterfall
LOCID = KTN, NAME = Ketchikan
LOCID = KXA, NAME = Kasaan
LOCID = HYL, NAME = Hollis
LOCID = CGA, NAME = Craig


* **Shape Field Tokens:**

![](https://wetlands.io/file/teaching/geog-503/images/cursor_shape_tokens.png)

In [7]:
import arcpy
import arcpy.da
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
fields = ["NAME","SHAPE@X","SHAPE@Y"]
cursor =  arcpy.da.SearchCursor(fc, fields)
for index, row in enumerate(cursor):
    print("NAME = {:20} X = {:.4f}    Y = {:.4f}".format(row[0],row[1],row[2]))
    if index == 10: 
        break
del cursor, row

NAME = Hyder                X = -130.0125    Y = 55.9043
NAME = Chignik Lagoon       X = -158.5326    Y = 56.3079
NAME = Koyuk                X = -161.1511    Y = 64.9381
NAME = Kivalina             X = -164.5256    Y = 67.7452
NAME = Ketchikan Harbor     X = -131.6851    Y = 55.3478
NAME = Metlakatla           X = -131.5781    Y = 55.1310
NAME = Waterfall            X = -133.2433    Y = 55.2963
NAME = Ketchikan            X = -131.7137    Y = 55.3556
NAME = Kasaan               X = -132.3975    Y = 55.5374
NAME = Hollis               X = -132.6461    Y = 55.4816
NAME = Craig                X = -133.1478    Y = 55.4788


## Using the InsertCursor 

* **General syntax:**
    - arcpy.da.InsertCursor(in_table, field_names)
    - <http://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-data-access/insertcursor-class.htm>
    
* **Parameters:**

![](https://wetlands.io/file/teaching/geog-503/images/insert_cursor_params.png)

* **Methods:**

![](https://wetlands.io/file/teaching/geog-503/images/insert_cursor_methods.png)

In [8]:
import arcpy
import arcpy.da
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
fields = ["NAME","SHAPE@X","SHAPE@Y"]
cursor =  arcpy.da.InsertCursor(fc, fields)
cursor.insertRow(["Binghamton",-155,60])
del cursor

In [9]:
import arcpy
import arcpy.da
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
fields = ["NAME","SHAPE@XY"]
cursor =  arcpy.da.InsertCursor(fc,fields)
cursor.insertRow(["Corning",(-154,55)])
del cursor

## Using the UpdateCursor 

* **General syntax:**
    - arcpy.da.UpdateCursor(in_table,field_names, (where_clause},{spatial_reference}, (explode_to_points},{sql_clause})
    - <http://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-data-access/updatecursor-class.htm>
    
* **Parameters:**

![](https://wetlands.io/file/teaching/geog-503/images/update_cursor_params.png)

* **Methods:**

![](https://wetlands.io/file/teaching/geog-503/images/update_cursor_methods.png)

* **updateRow example:**

In [10]:
import arcpy
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
fields = ["NAME"]
cursor =  arcpy.da.UpdateCursor(fc, fields)
for row in cursor:
    if row[0] == "Binghamton":
        row[0] = "Albany"
        cursor.updateRow(row)
del cursor, row

* **deleteRow example:**

In [11]:
import arcpy
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
fields = ["NAME"]
cursor =  arcpy.da.UpdateCursor(fc, fields)
for row in cursor:
    if row[0] == "Albany" or row[0] == "Corning":
        cursor.deleteRow()
del cursor, row

## Cursor Table Locking

* **Shared lock**
    - Placed on a table any time a table or feature class is accessed by ArcGIS.
    - Examples of shared locks: Displaying a feature class in ArcMap or previewing a table in ArcCatalog.

* **Exclusive lock**
    - Applied when changes are made to a table or feature class, either to field values or to the schema.
    - Examples of when an exclusive lock is applied: Using an UpdateCursor or InsertCursor in a Python IDE, saving edits to a feature class in ArcMap, or changing the schema in ArcCatalog.
    
* **Release the lock**
    - del row
    - del cursor

In [12]:
import arcpy
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
fields = ["NAME"]
cursor =  arcpy.da.UpdateCursor(fc, fields)
for row in cursor:
    if row[0] == "Albany":
        cursor.deleteRow()
del row
del cursor

## Using SQL in Python

* **Structured Query Language (SQL)**
    - Apply a query based on attribute table
    - As where_clause parameters in many tools in ArcToolbox

* **Example:**
    - SearchCursor(in_table, field_names {where_clause}, {spatial_reference}, {fields}, {explode_to_points})
    - Select_analysis(in_features, out_feature_class, {where_clause})

In [13]:
import arcpy
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
fields = ["NAME","LOCID"]
cursor =  arcpy.da.SearchCursor(fc, fields)
for row in cursor:
    if row[0] == "Goodnews":
        print("LOCID = {}".format(row[1]))
del cursor

LOCID = GNU


* **Field delimiters**
    - (" ") for shapefiles and file geodatabase feature classes
    - ([ ]) for personal geodatabase feature classes
    
* **AddFieldDelimiters function**
    - AddFieldDelimiters(datasource,field)
    
![](https://wetlands.io/file/teaching/geog-503/images/sql.png)

In [14]:
import arcpy
arcpy.env.workspace = r"C:\Geog503\data"
fc = "airports.shp"
fieldname = "NAME"
delimfield = arcpy.AddFieldDelimiters(fc,fieldname)
where_clause = delimfield + "='Goodnews'"
print(where_clause)
cursor =  arcpy.da.SearchCursor(fc,["NAME","LOCID"],where_clause)
for row in cursor:
    print("LOCID = {}".format(row[1]))
del cursor, row

"NAME"='Goodnews'
LOCID = GNU


## Validating Table Names

* **Determine whether a table name is valid for a workspace**
* **General syntax:**
    - arcpy.ValidateTableName(name, {workspace})
* **Any invalid characters will be replaced by an underscore (_)**


In [15]:
import arcpy
workspace = r"C:\Geog503\data\testdata.gdb"
table = arcpy.ValidateTableName("all roads",workspace)
print(table)

all_roads


In [16]:
import arcpy
import os
arcpy.env.workspace = r"C:\Geog503\data"
outworkspace =  r"C:\Geog503\data\testdata.gdb"
fclist = arcpy.ListFeatureClasses()
for fc in fclist:
    print(fc)
    fcname = arcpy.Describe(fc).basename
    newfcname = arcpy.ValidateTableName(fcname)
    outfc = os.path.join(outworkspace,newfcname)
    print(outfc)
    arcpy.CopyFeatures_management(fc,outfc)

airports.shp
C:\Geog503\data\testdata.gdb\airports
alaska.shp
C:\Geog503\data\testdata.gdb\alaska
roads.shp
C:\Geog503\data\testdata.gdb\roads


## Validating Field Names

* **Determine whether a field name is valid for a workspace**
* **General syntax:**
    - arcpy.ValidateFieldName(name, {workspace})
* **Any invalid characters will be replaced by an underscore (_)**

In [17]:
import arcpy
fc = r"C:\Geog503\data\airports.shp"
fieldname = arcpy.ValidateFieldName("NEW%&",fc)
arcpy.AddField_management(fc,fieldname,"TEXT","","",10)

<Result 'C:\\Geog503\\data\\airports.shp'>

In [18]:
fields = arcpy.ListFields(fc)
for field in fields:
    print(field.name)

FID
Shape
AREA
PERIMETER
AIRPRTX020
LOCID
FEATURE
NAME
TOT_ENP
STATE
COUNTY
FIPS
STATE_FIPS
NEW__


## Creating Unique Names

* **Creates a unique filename in the specified workspace by appending a number to the input name**

In [19]:
import arcpy
arcpy.env.workspace = r"C:\Geog503\data\Results"
fc = r"C:\Geog503\data\airports.shp"
unique_name = arcpy.CreateUniqueName("buffer.shp")
arcpy.Buffer_analysis(fc,unique_name,"1000 FEET")

<Result 'C:\\Geog503\\data\\Results\\buffer.shp'>

In [20]:
import arcpy
arcpy.env.workspace = r"C:\Geog503\data\Results"
fc = r"C:\Geog503\data\airports.shp"
unique_name = arcpy.CreateUniqueName("buffer.shp")
arcpy.Buffer_analysis(fc,unique_name,"1000 FEET")

<Result 'C:\\Geog503\\data\\Results\\buffer0.shp'>

## Parsing Table and Field Names

* **ParseTableName function**
    - Return database name, owner name, and table name.
    - Separated by a comma (,)

In [21]:
import arcpy
arcpy.env.workspace = r"C:\Geog503\data\Results"
fc = r"C:\Geog503\data\airports.shp"
fullname = arcpy.ParseTableName(fc)
namelist = fullname.split(", ")
databasename = namelist[0]
ownername = namelist[1]
fcname = namelist[2]
print(databasename)
print(ownername)
print(fcname)

(null)
(null)
C:\Geog503\data\airports.shp


## Working with Text Files

* **General syntax**
    - open(name, {mode}, {buffering})
    - <https://docs.python.org/2/tutorial/inputoutput.html#reading-and-writing-files>
* **Mode**
    - r: read mode
    - w: write mode
    - +: read/write mode 
    - b: binary mode others
    - a: append mode


In [22]:
f = open(r"C:\Geog503\data\Results\mytext.txt","w")
f.write("Binghamton University")
f.close()

###  Reading Text Files

* **General syntax**
    - open(name, "r", {buffering})

In [23]:
f = open(r"C:\Geog503\data\Results\mytext.txt")
print(f.read(10))
print(f.read(1))
print(f.read(10))
f.seek(0)
print(f.read())
f.close()

Binghamton
 
University
Binghamton University


###  Writing Text Files

* **General syntax**
    - open(name, "a", {buffering})

In [24]:
f = open(r"C:\Geog503\data\Results\mytext.txt","a")
f.write("\nGeography Dept")
f.write("\nGeog 503")
f.close()

In [25]:
f = open(r"C:\Geog503\data\Results\mytext.txt")
print(f.read())
f.close()

Binghamton University
Geography Dept
Geog 503


###  Reading Text Files

* **Three functions:**
    - read()
    - readline()
    - readlines()

In [26]:
f = open(r"C:\Geog503\data\Results\mytext.txt")
print(f.readline())
print(f.readline())
print(f.readline())
f.close()

Binghamton University

Geography Dept

Geog 503


In [27]:
f = open(r"C:\Geog503\data\Results\mytext.txt")
print(f.readlines())
f.close()

['Binghamton University\n', 'Geography Dept\n', 'Geog 503']


In [28]:
f = open(r"C:\Geog503\data\Results\mytext.txt")
lines = f.readlines()
f.close()
lines[2] = "GIS Programming"
f = open(r"C:\Geog503\data\Results\newtext.txt","w")
f.writelines(lines)
f.close()

In [29]:
f = open(r"C:\Geog503\data\Results\newtext.txt")
print(f.read())
f.close()

Binghamton University
Geography Dept
GIS Programming


In [30]:
f = open(r"C:\Geog503\data\Results\mytext.txt")
for line in f:
    print(line)
print("*********************")
f.seek(0)
while True:
    line = f.readline()
    if not line: break
    print(line)
print("*********************")
f.seek(0)
for line in f.readlines():
    print(line)
f.close()

Binghamton University

Geography Dept

Geog 503
*********************
Binghamton University

Geography Dept

Geog 503
*********************
Binghamton University

Geography Dept

Geog 503


### Replacing Text Contents

In [31]:
in_file = open(r"C:\Geog503\data\Results\mytext.txt")
out_file = open(r"C:\Geog503\data\Results\newtext.txt","w")
for line in in_file:
    str = line.replace("Binghamton University","State University of New York at Binghamton")
    out_file.write(str)
in_file.close()
out_file.close()

In [32]:
f = open(r"C:\Geog503\data\Results\newtext.txt")
print(f.read())
f.close()

State University of New York at Binghamton
Geography Dept
Geog 503
